In [ ]:
import re 
import math 
#import eli5

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 

import sklearn 
#from eli5.sklearn import PermutationImportance
from urllib.parse import urlparse 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier

#import tensorflow as tf
#from tensorflow.keras import layers
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.callbacks import ReduceLROnPlateau

#from sklearn.inspection import PartialDependenceDisplay

#import joblib

In [ ]:
PATH = '../data/test_dataset/CSIC2010/csic_database.csv'
df = pd.read_csv(PATH)
display(df.head())

In [ ]:
print(f"Number of samples: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
sns.set_style("darkgrid")
sns.countplot(data=df, x='Unnamed: 0')

Dropping samples with at least 1 NaN value will make to lose all the other Request Methods besides POST, this option is discarded since dropping data is not usually a good choice

In [ ]:
feature_name = df.columns
print(feature_name)

In [ ]:
total_features = [
  'Unnamed: 0', 
  'Method', 
  'User-Agent', 
  'Pragma', 
  'Cache-Control',
  'Accept', 
  'Accept-encoding', 
  'Accept-charset', 
  'language', 
  'host',
  'cookie', 
  'content-type', 
  'connection', 
  'lenght', 
  'content',
  'classification', 
  'URL'
]

In [ ]:
X = df[total_features]
print(X)

#### Removing not discriminatory features
Enumerating unique values for each feature

In [ ]:
# Removing not discriminatory features and making some adjustments on feature names
X = X.rename(columns={'Unnamed: 0': 'Class'})
X = X.rename(columns={'lenght': 'content_length'})

feature_names = [
  'Class',
  'Method',
  'host',
  'cookie',
  'Accept', 
  'content_length', 
  'content',
  'classification',
  'URL'
]
X = X[feature_names]
print(X)

In [ ]:
y=X.Class
print(y)

In [ ]:
size = X.shape[1]
# Get list of categorical variables
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

print(f"Categorical variables: {object_cols}")

Load Models

#### Pre-Processing on the feature: Content Length

In [ ]:
print(X.content_length)

Operations on the 'content_lenght' feature

In [ ]:
# replace NaN values with 0
# removing the 'content-lenght' string and keeping only the numerical value 
X['content_length'] = X['content_length'].astype(str)
X['content_length'] = X['content_length'].str.extract(r'(\d+)')
X['content_length'] = pd.to_numeric(X['content_length'], errors='coerce').fillna(0)
print(X.content_length)

GET methods have the content_length set to 0 since they where all NaN (this method does not have to provide content)



In [ ]:
filtered_length = X.loc[X['Method'] == 'GET', 'content_length']
print(filtered_length)


URL PRE-PROCESSING

In [ ]:
url_counts = X['URL'].value_counts()
most_common_urls = url_counts.head(10)  # Extract the top 10 most common strings

print("Most common URLs:")
for i, (url, count) in enumerate(most_common_urls.items(), 1):
  print(f"{i}. URL: {url} - Count: {count}")

Utils for URL/Content pre-processing

In [ ]:
def count_dot(url):
  count_dot = url.count('.')
  return count_dot

def no_of_dir(url):
  urldir = urlparse(url).path
  return urldir.count('/')

def no_of_embed(url):
  urldir = urlparse(url).path
  return urldir.count('//')

def shortening_service(url):
  match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                    'tr\.im|link\.zip\.net',
                    url)
  if match:
    return 1
  else:
    return 0

def count_http(url):
  return url.count('http')

def count_per(url):
  return url.count('%')

def count_ques(url):
  return url.count('?')

def count_hyphen(url):
  return url.count('-')

def count_equal(url):
  return url.count('=')

def url_length(url):
  return len(str(url))

#Hostname Length
def hostname_length(url):
  return len(urlparse(url).netloc)

import re

def suspicious_words(url):
  score_map = {
    'error': 30,
    'errorMsg': 30,
    'id': 10,
    'errorID': 30,
    'SELECT': 50,
    'FROM': 50,
    'WHERE': 50,
    'DELETE': 50,
    'USERS': 50,
    'DROP': 50,
    'CREATE': 50,
    'INJECTED': 50,
    'TABLE': 50,
    'alert': 30,
    'javascript': 20,
    'cookie': 25,
    '--': 30,
    '.exe': 30,
    '.php': 20,
    '.js': 10,
    'admin': 10,
    'administrator': 10,
    '\'': 30,
    'password': 15,
    'login': 15,
    'incorrect': 20,
    'pwd': 15,
    'tamper': 25,
    'vaciar': 20,
    'carrito': 25,
    'wait': 30,
    'delay': 35,
    'set': 20,
    'steal': 35,
    'hacker': 35,
    'proxy': 35,
    'location': 30,
    'document.cookie': 40,
    'document': 20,
    'set-cookie': 40,
    'create': 40,
    'cmd': 40,
    'dir': 30,
    'shell': 40,
    'reverse': 30,
    'bin': 20,
    'cookiesteal': 40,
    'LIKE': 30,
    'UNION': 35,
    'include': 30,
    'file': 20,
    'tmp': 25,
    'ssh': 40,
    'exec': 30,
    'cat': 25,
    'etc': 30,
    'fetch': 25,
    'eval': 30,
    'wait': 30,
    'malware': 45,
    'ransomware': 45,
    'phishing': 45,
    'exploit': 45,
    'virus': 45,
    'trojan': 45,
    'backdoor': 45,
    'spyware': 45,
    'rootkit': 45,
    'credential': 30,
    'inject': 30,
    'script': 25,
    'iframe': 25,
    'src=': 25,
    'onerror': 30,
    'prompt': 20,
    'confirm': 20,
    'eval': 25,
    'expression': 30,
    'function\(': 20,
    'xmlhttprequest': 30,
    'xhr': 20,
    'window.': 20,
    'document.': 20,
    'cookie': 25,
    'click': 15,
    'mouseover': 15,
    'onload': 20,
    'onunload': 20,
  }

  matches = re.findall(r'(?i)' + '|'.join(score_map.keys()), url)

  total_score = sum(score_map.get(match.lower(), 0) for match in matches)
  return total_score

def digit_count(url):
  digits = 0
  for i in url:
    if i.isnumeric():
      digits = digits + 1
  return digits

def letter_count(url):
  letters = 0
  for i in url:
    if i.isalpha():
      letters += 1
  return letters

def count_special_characters(url):
  special_characters = re.sub(r'[a-zA-Z0-9\s]', '', url)
  count = len(special_characters)
  return count

# Number of Parameters in URL
def number_of_parameters(url):
  params = urlparse(url).query
  return 0 if params == '' else len(params.split('&'))

# Number of Fragments in URL
def number_of_fragments(url):
  frags = urlparse(url).fragment
  return len(frags.split('#')) - 1 if frags == '' else 0

# URL is Encoded
def is_encoded(url):
  return int('%' in url.lower())

def unusual_character_ratio(url):
  total_characters = len(url)
  unusual_characters = re.sub(r'[a-zA-Z0-9\s\-._]', '', url)
  unusual_count = len(unusual_characters)
  ratio = unusual_count / total_characters if total_characters > 0 else 0
  return ratio

In [ ]:
X['count_dot_url'] = X['URL'].apply(count_dot)
X['count_dir_url'] = X['URL'].apply(no_of_dir)
X['count_embed_domain_url'] = X['URL'].apply(no_of_embed)
X['short_url'] = X['URL'].apply(shortening_service)
X['count-http'] = X['URL'].apply(count_http)
X['count%_url'] = X['URL'].apply(count_per)
X['count?_url'] = X['URL'].apply(count_ques)
X['count-_url'] = X['URL'].apply(count_hyphen)
X['count=_url'] = X['URL'].apply(count_equal)
X['hostname_length_url'] = X['URL'].apply(hostname_length)
X['sus_url'] = X['URL'].apply(suspicious_words)
X['count-digits_url'] = X['URL'].apply(digit_count)
X['count-letters_url'] = X['URL'].apply(letter_count)
X['url_length'] = X['URL'].apply(url_length)
X['number_of_parameters_url'] = X['URL'].apply(number_of_parameters)
X['number_of_fragments_url'] = X['URL'].apply(number_of_fragments)
X['is_encoded_url'] = X['URL'].apply(is_encoded)
X['special_count_url'] = X['URL'].apply(count_special_characters)
X['unusual_character_ratio_url'] = X['URL'].apply(unusual_character_ratio)


In [ ]:
# Select the features and class variable for plotting
new_features = ['count_dot_url', 'count_dir_url', 'count_embed_domain_url', 'count-http',
                'count%_url', 'count?_url', 'count-_url', 'count=_url', 'url_length', 'hostname_length_url',
                'sus_url', 'count-digits_url', 'count-letters_url', 'number_of_parameters_url',
                'number_of_fragments_url', 'is_encoded_url','special_count_url','unusual_character_ratio_url']

# Create a DataFrame with the selected features
set = X[new_features]

for new_feature in X.columns:
  if new_feature in X.columns:
    unique_count = X[new_feature].nunique()
    print(f"Number of unique values for {new_feature}: {unique_count}")
  else:
    print(f"Column '{new_feature}' does not exist in the DataFrame.")

In [ ]:
# Removing Cookies as Feature 
## cookies are unique for each sample, this feature cannot be used as discriminant 
unique_count = X['cookie'].nunique()
print(f"Count of unique values in 'cookie': {unique_count}")

#### Encoding categorical features

In [ ]:
X['Accept'] = X['Accept'].astype(str)
X['Accept'] = X['Accept'].str.extract(r'(\d+)')
X['Accept'] = pd.to_numeric(X['Accept'], errors='coerce').fillna(1)

In [ ]:
lb_make = LabelEncoder()
X["Method_enc"] = lb_make.fit_transform(X["Method"])
X["host_enc"] =lb_make.fit_transform(X["host"])
X["Accept_enc"] =lb_make.fit_transform(X["Accept"])

In [ ]:
unique_count_met = X["Method_enc"].nunique()
unique_count_host = X["host_enc"].nunique()
unique_count_acc = X["Accept_enc"].nunique()


print(f"Number of unique values for 'Method_enc': {unique_count_met}")
print(f"Number of unique values for 'host_enc': {unique_count_host}")
print(f"Number of unique values for 'Accept_enc': {unique_count_acc}")


In [ ]:
X.head()

In [ ]:
X.tail()

In [ ]:
def apply_to_content(content,function):
  if pd.isna(content):
    return 0
  elif isinstance(content, str):
    return function(content)

#"""
#                'count_dot_content','count_dir_content','count_embed_domain_content','count%_content','count?_content',
#               'count-_content','count=_content','hostname_length_content','sus_content','count_digits_content',
#              'count_letters_content','content_length','number_of_parameters_content','number_of_fragments_content',
#             'is_encoded_content','special_count_content','unusual_character_ratio_content'
#            ]"""

X['count_dot_content'] = X['content'].apply(apply_to_content, function=count_dot)
X['count_dir_content'] = X['content'].apply(apply_to_content, function=no_of_dir)
X['count_embed_domain_content'] = X['content'].apply(apply_to_content, function=no_of_embed)
X['count%_content'] = X['content'].apply(apply_to_content, function=count_per)
X['count?_content'] = X['content'].apply(apply_to_content, function=count_ques)
X['count-_content'] = X['content'].apply(apply_to_content, function=count_hyphen)
X['count=_content'] = X['content'].apply(apply_to_content, function=count_equal)
X['content_length'] = X['content'].apply(apply_to_content, function=url_length)
X['sus_content'] = X['content'].apply(apply_to_content, function=suspicious_words)
X['count_digits_content'] = X['content'].apply(apply_to_content, function=digit_count)
X['count_letters_content'] = X['content'].apply(apply_to_content, function=letter_count)
X['special_count_content'] = X['content'].apply(apply_to_content, function=count_special_characters)
X['is_encoded_content'] = X['content'].apply(apply_to_content, function=is_encoded)
#X['unusual_character_ratio_content'] = X['content'].apply(apply_to_content, function=unusual_character_ratio)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select the features and class variable for plotting
new_content_features = [
  'count_dot_content', 'count_dir_content', 
  'count_embed_domain_content', 'count%_content', 
  'count?_content', 'count-_content', 
  'count=_content', 'sus_content', 
  'count_digits_content', 'count_letters_content', 
  'content_length', 'is_encoded_content', 
  'special_count_content'
]

# Create a DataFrame with the selected features
selected_features_df = X[new_content_features]

for feature_name in selected_features_df.columns:
  if feature_name in X.columns:
    unique_count = selected_features_df[feature_name].nunique()
    print(f"Number of unique values for {feature_name}: {unique_count}")
  else:
    print(f"Column '{feature_name}' does not exist in the DataFrame.")


In [ ]:
X.columns

#### Building the final dataset to use for the classification

In [ ]:
X.columns

In [ ]:
labels=['count_dot_url', 'count_dir_url', 'count_embed_domain_url', 'count-http',
        'count%_url', 'count?_url', 'count-_url', 'count=_url', 'url_length', 'hostname_length_url',
        'sus_url', 'count-digits_url', 'count-letters_url', 'number_of_parameters_url',
        'is_encoded_url','special_count_url','unusual_character_ratio_url',
        #method
        'Method_enc',
        #content
        'count_dot_content','count%_content',
        'count-_content','count=_content','sus_content','count_digits_content',
        'count_letters_content','content_length',
        'is_encoded_content','special_count_content']
print(X[labels])

In [ ]:
y=X['classification']
print(y)

In [ ]:
print('computing...)')
#split dataset in test and train 
x_tr, x_ts, y_tr, y_ts = train_test_split(X[labels], y, test_size=0.3, random_state=0)

print('Done!')

In [ ]:
x_tr.head(5)


In [ ]:
x_tr.tail(5)


#### Classifiers

In [ ]:
random_forest_model = RandomForestClassifier(random_state=1000)
print('Computing....')
# Fit the model
random_forest_model.fit(x_tr,y_tr)
print('Done!')


In [ ]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(random_forest_model, threshold="mean")
X_train_selected = selector.transform(x_tr)
X_test_selected = selector.transform(x_ts)

svm = SVC(kernel='rbf', C=2, gamma='scale')  # You can tune these hyperparameters
svm.fit(X_train_selected, y_tr)


In [ ]:
y_pred = svm.predict(X_test_selected)
accuracy = accuracy_score(y_ts, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_ts, y_pred))


In [ ]:
RT_predictions= random_forest_model.predict(x_ts)
print('MAE', mean_absolute_error(y_ts, RT_predictions))
print("Accuracy", accuracy_score(y_ts, RT_predictions))
print("Precision", precision_score(y_ts, RT_predictions, average='weighted', labels=np.unique(RT_predictions)))
print("Recall", recall_score(y_ts, RT_predictions, average='weighted', labels=np.unique(RT_predictions)))
print("F1", f1_score(y_ts, RT_predictions, average='weighted', labels=np.unique(RT_predictions)))
print("ROC AUC", roc_auc_score(y_ts, RT_predictions, average='weighted', labels=np.unique(RT_predictions)))
error_rt = (RT_predictions != y_ts).mean()
print("Test error: {:.1%}".format(error_rt))


# save the model 
filename = 'APPLICATION LAYER.sav'
joblib.dump(random_forest_model, open(filename, 'wb')) 


In [ ]:
print(y_tr.unique())
print(y_tr.name)


In [ ]:
x_ts = x_ts.reset_index(drop=True)
y_ts = y_ts.reset_index(drop=True)

for k in range(np.unique(y_ts).size):
  print('mean of class ' + str(k) + ':\n', x_ts[y_ts == k].mean(axis=0))


In [ ]:
print(classification_report(y_ts, RT_predictions, target_names = ['Normal (class 0)','Anomalous (class 1)']))


In [ ]:
label = ['Normal', 'Anomalous']
cm = confusion_matrix(y_ts, RT_predictions)
cm = pd.DataFrame(cm, index=['0', '1'], columns=['0', '1'])

plt.figure(figsize=(10, 10))
sns.heatmap(cm, cmap="Blues", linecolor='black', linewidth=1, annot=True, fmt='', xticklabels=label, yticklabels=label)
plt.title("Random Forest")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
DT_model = DecisionTreeClassifier(random_state=2)
print('Computing....')
DT_model.fit(x_tr,y_tr)
print('Done!')

In [ ]:
DT_predictions= DT_model.predict(x_ts)
print('MAE', mean_absolute_error(y_ts, DT_predictions))
print("Accuracy", accuracy_score(y_ts, DT_predictions))
print("Precision", precision_score(y_ts, DT_predictions, average='weighted', labels=np.unique(DT_predictions)))
print("Recall", recall_score(y_ts, DT_predictions, average='weighted', labels=np.unique(DT_predictions)))
print("F1", f1_score(y_ts, DT_predictions, average='weighted', labels=np.unique(DT_predictions)))
print("ROC AUC", roc_auc_score(y_ts, DT_predictions, average='weighted', labels=np.unique(DT_predictions)))
error_dt = (DT_predictions != y_ts).mean()
print("Test error: {:.1%}".format(error_dt))


In [ ]:
cm = confusion_matrix(y_ts,DT_predictions)
cm = pd.DataFrame(cm , index = ['0','1'] , columns = ['0','1'])
plt.title("Decision Tree")
plt.xlabel("Predicted")
plt.ylabel("Actual")
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='',xticklabels = label,yticklabels = label)
